# Evaluation on LFW

The aim of this notebook is to evaluate the clustering algorithm on the Labeled Faces In the Wild dataset which can be  
found [here](http://vis-www.cs.umass.edu/lfw/)  
The dataset has about 13,233 images of 5749 people where there are 1680 people with two or more images.

Before running this notebook it is essential to run the `embedder.py`. On doing so all the embeddings for faces in the dataset are computed and saved into a pickle file. This notebook loads that pickle file and runs the clustering algorithm 

## Import Necessary Libraries

In [1]:
import pickle
import numpy as np
import networkx as nx
import os
import shutil
import math
from random import shuffle
import pprint
pp = pprint.PrettyPrinter()

#### Importing chinese whispers algorithm
As we have already defined the clustering algorithm in `clusterer.py` we can directly import it

In [2]:
from CW import draw_graph
from CW import chinese_whispers

The embeddings for the faces in the dataset (LFW in this case) should be loaded. The embeddings can be computed from the script
`embedder.py`

In [3]:
# Define function for nCr
def nCr(n,r):
    fact = math.factorial
    return fact(n)/(fact(r)*fact(n-r))
# Partially view a dictionary with pretty printing
def partial_dict_view(dictionary,n):
    pp.pprint({k: v for i, (k, v) in enumerate(dictionary.items()) if i < n})

## Run Chinese Whispers on the embeddings

In [4]:
data = pickle.load(open("embeddings.pickle","rb"))

Since the embeddings are loaded we can then create the graph

In [5]:
graph = draw_graph(data,0.67)
graph = chinese_whispers(graph,30)
# Takes about 5 minutes for this dataset

Iterations: 100%|██████████████████████████████████████████████████████████████████████| 30/30 [00:34<00:00,  1.15s/it]


## Calculate Evaluation Metric

To evaluate the clustering we need to define a few things.
First we calculate true pairwise positives,true pairwise negatives, false pairwise positives false pairwise negatives by compairing pairs of nodes.
For a cluster the correct decisions are when:
- **true pairwise positives (TP)**: when two nodes belonging to same class also belong to same cluster
- **true pairwise negatives (TN)**: when two nodes of different classes belong to different clusters
and incorrect decisions are:
- **false pairwise positives (FP)**: when two nodes of different classes belong to same cluster
- **false pairwise negatives (FN)**: when two nodes of same class belong to different clusters

So in case of clustering instead of looking at individual data points, pairs of datapoints are studied

Then we calculate the precision and recall as:  
  
$Precision = \frac{TP}{TP+FP} $  
  
$Recall = \frac{TP}{TP+FN}$

The evaluation metric used is the F-score.  
F-score is defined as the harmonic mean of precision and recall.

$ F-measure =  \frac{2*Precision*Recall}{Precision+Recall}$

We will first calculate the precision and recall using True positives, False positives and False negatives

Firstly create a dictionary which maps a cluster to the number of images it contains.  
**NOTE:** a lot of people in the LFW dataset have only one image of them, hence there can be many clusters with only one image

In [6]:
cluster_to_num_images = {}
for node in graph.nodes:
    if graph.nodes[node]['cluster'] in cluster_to_num_images:
        cluster_to_num_images[graph.nodes[node]['cluster']] += 1
    else:
        cluster_to_num_images[graph.nodes[node]['cluster']] = 1

In [7]:
partial_dict_view(cluster_to_num_images,10)

{'Person 1': 1,
 'Person 10': 2,
 'Person 11159': 2,
 'Person 12187': 2,
 'Person 13905': 8,
 'Person 14': 1,
 'Person 3': 1,
 'Person 5439': 4,
 'Person 6': 4,
 'Person 8': 1}


Now let's compute the total positives  
  
The number of pairs formed within a cluster having $n$ nodes is given by ${n \choose 2}$

In [8]:
total_pw_positives = 0
for cluster,num_images in cluster_to_num_images.items():
    # It's a positive only if a pair can be formed
    if num_images >= 2:
        total_pw_positives += nCr(num_images,2)

In [9]:
print(total_pw_positives)

265384.0


- The subdirectory of the image in which it resides in the LFW folder is the identity of the person. 
- In this scenario the identity is the class.
- Hence the  classes can be taken from the subdirectory in the path of the image.
- The nodes of the networkx graph contain an attribute which has the relative path for the image.
- For example `lfw\\Aaron_Eckhart\\Aaron_Eckhart_0001.jpg` is the image corresponding to the person named Aaron Eckhart

Firstly we create a dictionary which maps a cluster to a dictionary which describes the cluster. i.e the latter's value is a dictionary which maps each class (identity) in the cluster to the number of nodes (image embeddings) in the cluster which belong to that class

In [10]:
clusters = list(cluster_to_num_images.keys())

In [11]:
cluster_to_desc = {}
for cluster in clusters:
    cluster_to_desc[cluster] = {}
    for node in graph.nodes:
        if graph.nodes[node]['cluster'] == cluster:
            
            path = os.path.normpath(graph.nodes[node]['source'])
            class_ = path.split(os.sep)[1]
            # find a cleaner implementation
            if class_ not in cluster_to_desc[cluster]:
                cluster_to_desc[cluster][class_] = 1
            else:
                cluster_to_desc[cluster][class_] += 1

In [12]:
# Detailed description of the dictionary
partial_dict_view(cluster_to_desc,5)

{'Person 1': {'Aaron_Eckhart': 1},
 'Person 12187': {'Aaron_Guiel': 1, 'Shane_Phillips': 1},
 'Person 3': {'Aaron_Patterson': 1},
 'Person 6': {'Aaron_Peirsol': 4},
 'Person 8': {'Aaron_Pena': 1}}


Now we can compute the true pairwise positives.    
True pairwise positives are basically the number of pairs formed between two nodes in the cluster
which belong to the same class.

For example in the dictionary above the class Aaron Peirsol has 4 images of him in cluster number 7.  
  
Hence ${4 \choose 2}$ pairs (true pairwise positives) can be formed

In [13]:
true_pw_positives = 0
for cluster in cluster_to_desc.keys():
    for class_ in cluster_to_desc[cluster].keys():
        if cluster_to_desc[cluster][class_] >= 2:
            true_pw_positives += nCr(cluster_to_desc[cluster][class_],2)

In [14]:
print(true_pw_positives)

238554.0


False positives can then be calculated from subtracting from the total pairwise positives.  
  
$FP = Total Positives - TP$

In [15]:
false_pw_positives = total_pw_positives - true_pw_positives
print(false_pw_positives)

26830.0


Now let's create a dictionary which maps a class(identity) to the number of images it contains. 

In [16]:
class_to_num_images = {}
for class_ in os.listdir("lfw"):
    num_images = 0
    for image in os.listdir(os.path.join("lfw",class_)):
        num_images += 1
    class_to_num_images[class_] = num_images

In [17]:
partial_dict_view(class_to_num_images,10)

{'Aaron_Eckhart': 1,
 'Aaron_Guiel': 1,
 'Aaron_Patterson': 1,
 'Aaron_Peirsol': 4,
 'Aaron_Pena': 1,
 'Aaron_Sorkin': 2,
 'Aaron_Tippin': 1,
 'Abba_Eban': 1,
 'Abbas_Kiarostami': 1,
 'Abdel_Aziz_Al-Hakim': 1}


We can then calculate the false negatives using the above dictionary

In [18]:
false_pw_negatives = 0
# Iterate through all classes
for class_ in class_to_num_images.keys():
    prev_occurence = 0
    for cluster in cluster_to_num_images.keys():
        if class_ in cluster_to_desc[cluster]:
            # Get the number of nodes for the current class in the cluster
            num_of_class_in_cluster = cluster_to_desc[cluster][class_]
            
            # Get the number of nodes for the current class not in the cluster
            # It can be calculated by subtracting the number of times a class occurs in the cluster
            # from the total number of times it occurs in the dataset
            num_of_class_out_of_cluster = class_to_num_images[class_] - num_of_class_in_cluster
            
            # The number of pairs formed containing mismatched nodes is the number of nodes in
            # the cluster multiplied with the number out of the cluster
            # To account for pairs added by the previous cluster we subtract 'prev'
            false_pw_negatives += num_of_class_in_cluster*(num_of_class_out_of_cluster-prev_occurence)
            prev_occurence += num_of_class_in_cluster
            

In [19]:
print(false_pw_negatives)

1054


Finally with all these values we can calculate the F-measure.  
  
$ F-measure =  \frac{2*Precision*Recall}{Precision+Recall}$

In [23]:
precision = true_pw_positives / (true_pw_positives + false_pw_positives)

recall = true_pw_positives / (true_pw_positives + false_pw_negatives)

f_measure = (2*precision*recall)/(precision + recall)

print("Precision: {:.3f}".format(precision))
print("Recall: {:.3f}".format(recall))
print("F-measure: {:.3f}".format(f_measure))

Precision: 0.899
Recall: 0.996
F-measure: 0.945


### Precision vs Recall

Note that with a threshold value of `0.65` we were able to get a higher recall. Increasing the threshold value results in a higher precision but a lower recall. Since the task was to get all the images corresponding to a person in a corpus of photos a higher recall value was preferred.

There are 5749 people in the lfw dataset. Let's check number of people (clusters) the algorithm gives

In [21]:
print(len(clusters))

6090


Although the LFW dataset is divided into 5749 subdirectories each indicating a single person, there are many images where there are more than one face. But for these types of images only a single label is used. This can either result in two scenarios:
1. The identity of the extra face detected is nowhere present in the dataset. This can result in a seperate cluster. This results in a slightly higher number of clusters formed (close to 6000 in this case). 
  
2. The identity of the extra face detected in the image is present in the dataset, but the label corresponding to the image is different. In this case the face is assigned to the respective cluster, but while evaluating using this notebook it might be labeled as a false positive.

But even with these constraints the algorithm seems to have good results

**NOTE**: Different precision, recall and f-measure is obtained for every run of the clustering algorithm